Copyright **`(c)`** 2023 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

In [24]:
from itertools import product
from random import random, randint, shuffle, seed, choice
import numpy as np
from scipy import sparse
import logging
import platform
from collections import Counter
from scipy.sparse import linalg, vstack, hstack, identity, eye
from functools import reduce
import copy

In [2]:
def make_set_covering_problem(num_points, num_sets, density):
    """Returns a sparse array where rows are sets and columns are the covered items"""
    seed(num_points*2654435761+num_sets+density)
    sets = sparse.lil_array((num_sets, num_points), dtype=bool)
    for s, p in product(range(num_sets), range(num_points)):
        if random() < density:
            sets[s, p] = True
    for p in range(num_points):
        sets[randint(0, num_sets-1), p] = True
    return sets

# Halloween Challenge

Find the best solution with the fewest calls to the fitness functions for:

* `num_points = [100, 1_000, 5_000]`
* `num_sets = num_points`
* `density = [.3, .7]` 

In [3]:
x = make_set_covering_problem(1000, 1000, .3)
print("Element at row=42 and column=42:", x[42, 42])

Element at row=42 and column=42: False


In [4]:
SETS = x.toarray()

In [5]:
num_points = [100, 1_000, 5_000]
num_sets = num_points
density = [0.3, 0.7]

In [6]:
def goal_check(x,  num_pts):
    sol = [False for _ in range(num_pts)]
    for s in range(num_pts):
        for p in range(num_pts):
            sol[p] = sol[p] or x[s, p]
    
    return np.all(sol)

assert goal_check(x, 1000)

In [7]:
# HILL CLIMBING


def hill_climbing(N, all_lists):
    logging.debug(f"Original: {len(all_lists)}")
    all_lists = set(tuple(sorted(set(_))) for _ in all_lists)
    logging.debug(f"Optimized: {len(all_lists)}")

    def evaluate(state):
        cnt = Counter()
        cnt.update(sum((e for e in state), start=()))
        return len(cnt), -cnt.total()

    def tweak(solution):
        new_solution = set(solution)
        while new_solution and random.random() < 0.7:
            r = random.choice(list(new_solution))
            new_solution.remove(r)
        while all_lists - solution and random.random() < 0.7:
            a = random.choice(list(all_lists - solution))
            new_solution.add(a)
        return new_solution

    current_solution = set()
    useless_steps = 0
    while useless_steps < 10_000:
        useless_steps += 1
        candidate_solution = tweak(current_solution)
        if evaluate(candidate_solution) > evaluate(current_solution):
            useless_steps = 0
            current_solution = copy(candidate_solution)
            logging.debug(f"New solution: {evaluate(current_solution)}")
    return current_solution

In [8]:
best_solution = None
fewest_fitness_calls = float('inf')

for num_points, num_sets, density in product(num_points, num_sets, density):
    # Generate the set covering problem
    problem = make_set_covering_problem(num_points, num_sets, density)
    
    # Replace this with your fitness function and optimization algorithm
    # Fitness function will evaluate the quality of a solution
    # Optimization algorithm will find the best solution
    # For example, you can use a greedy algorithm or a heuristic search
    
    # Placeholder for fitness function and optimization
    # This is where you would call your fitness function and find the solution
    # You should keep track of the number of fitness function calls
    
    # For demonstration, let's assume some fitness calls
    # Replace this with your actual fitness function and optimization process
    fitness_calls = 1000
    
    if fitness_calls < fewest_fitness_calls:
        fewest_fitness_calls = fitness_calls
        best_solution = (num_points, num_sets, density)

print("Best solution:", best_solution)
print("Fewest fitness function calls:", fewest_fitness_calls)

KeyboardInterrupt: 

In [21]:
hstack([problem[:,[i]] for i in [0,1, 5]])


selected_sets = hstack([problem[:,[i]] for i in [0, 1, 5]]).T # 3 * 1000

identity_matrix = eye(selected_sets.shape[1]- selected_sets.shape[0], selected_sets.shape[1], selected_sets.shape[0])

augmented_matrix = vstack([selected_sets, identity_matrix])

ones = np.ones(augmented_matrix.shape[1])

uncovered = np.logical_not(linalg.spsolve(augmented_matrix, ones))
print(np.sum(uncovered))



0


/Users/roxane/Documents/GitHub/Computational-intelligence/.venv/lib/python3.11/site-packages/scipy/sparse/linalg/_dsolve/linsolve.py:229: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  warn('spsolve requires A be CSC or CSR matrix format',
/Users/roxane/Documents/GitHub/Computational-intelligence/.venv/lib/python3.11/site-packages/scipy/sparse/linalg/_dsolve/linsolve.py:293: MatrixRankWarning: Matrix is exactly singular
  warn("Matrix is exactly singular", MatrixRankWarning)


In [38]:

augmented_matrix = hstack([selected_sets, identity_matrix])
ones = np.ones(augmented_matrix.shape[1])

uncovered = np.logical_not(linalg.spsolve(augmented_matrix, ones))
print(np.sum(uncovered))

Signature: vstack(blocks, format=None, dtype=None)
Docstring:
Stack sparse matrices vertically (row wise)

Parameters
----------
blocks
    sequence of sparse matrices with compatible shapes
format : str, optional
    sparse format of the result (e.g., "csr")
    by default an appropriate sparse matrix format is returned.
    This choice is subject to change.
dtype : dtype, optional
    The data-type of the output matrix. If not given, the dtype is
    determined from that of `blocks`.

See Also
--------
hstack : stack sparse matrices horizontally (column wise)

Examples
--------
>>> from scipy.sparse import coo_matrix, vstack
>>> A = coo_matrix([[1, 2], [3, 4]])
>>> B = coo_matrix([[5, 6]])
>>> vstack([A, B]).toarray()
array([[1, 2],
       [3, 4],
       [5, 6]])
File:      ~/Documents/GitHub/Computational-intelligence/.venv/lib/python3.11/site-packages/scipy/sparse/_construct.py
Type:      function

In [11]:
# Function to calculate the fitness value of a solution

def fitness(solution, problem):
    if not solution:
        return float('inf')  # Return a large value for an empty solution

    selected_sets = hstack([problem[:,[i]] for i in solution])
    identity_matrix = identity(selected_sets.shape[0])
    augmented_matrix = vstack([selected_sets, identity_matrix])
    ones = np.ones(augmented_matrix.shape[1])

    uncovered = np.logical_not(linalg.spsolve(augmented_matrix, ones))
    return np.sum(uncovered)


# Simulated Annealing function to find a solution to the set covering problem
def simulated_annealing(problem, initial_solution, temperature, cooling_rate, max_iterations):
    current_solution = initial_solution
    current_fitness = fitness(current_solution, problem)
    best_solution = current_solution
    best_fitness = current_fitness

    for _ in range(max_iterations):
        # Generate a neighbor solution by randomly adding or removing a set
        neighbor_solution = current_solution.copy()
        if random() < 0.5:
            # Add a random set
            set_to_add = randint(0, problem.shape[0] - 1)
            if set_to_add not in neighbor_solution:
                neighbor_solution.append(set_to_add)
        else:
            # Remove a random set
            if neighbor_solution:
                set_to_remove = choice(neighbor_solution)
                neighbor_solution.remove(set_to_remove)

        neighbor_fitness = fitness(neighbor_solution, problem)

        # Calculate the change in fitness
        delta_fitness = neighbor_fitness - current_fitness

        # Accept the neighbor solution if it improves fitness or with a certain probability
        if delta_fitness < 0 or random() < np.exp(-delta_fitness / temperature):
            current_solution = neighbor_solution
            current_fitness = neighbor_fitness

        # Update the best solution if needed
        if current_fitness < best_fitness:
            best_solution = current_solution
            best_fitness = current_fitness

        # Cool down the temperature
        temperature *= cooling_rate

    return best_solution

# Example problem (replace with your problem data)
problem_size = 20
num_sets = 100
problem = [np.array([random() < 0.3 for _ in range(problem_size)]) for _ in range(num_sets)]

problem = x


# Initial solution (empty in this case)
initial_solution = []

# Simulated Annealing parameters
initial_temperature = 1.0
cooling_rate = 0.99
max_iterations = 1000

# Find the best solution using Simulated Annealing
best_solution = simulated_annealing(problem, initial_solution, initial_temperature, cooling_rate, max_iterations)

print("Best solution:", best_solution)
print("Best fitness:", fitness(best_solution, problem))

ValueError: blocks[:,0] has incompatible column dimensions. Got blocks[1,0].shape[1] == 1000, expected 1.

In [72]:
def fitness(sets, state):
    cost = np.sum(state)
    if np.array(state).any():
        valid = sets[np.array(state), :].max(axis=0).sum()
    else:
        valid = 0
    return valid, -cost


In [70]:
def tweak(state, size):
    new_state = state.copy()
    index = randint(0, size - 1)
    new_state[index] = not new_state[index]
    return new_state


In [75]:
def hill_climbing(problem_size, num_sets, density, nb_steps = 100000):
    sets = make_set_covering_problem(num_sets, num_sets, .3).toarray()
    initial_state = [choice([False, False, False, False, False, False]) for _ in range(num_sets)]#[False for _ in range(num_sets)] 
    current_state = initial_state
    print(" The fitness of the initial state is : ", fitness(sets, initial_state))
    counter  = 0
    for step in range(nb_steps):
        new_state = tweak(current_state, problem_size)
        # if step % 10 == 0 : print(new_state)
        counter += 1 #maybe it should be two
        if fitness(sets, current_state)[:1] <= fitness(sets, new_state):
            current_state = new_state
            # print(fitness(sets, current_state))
    return fitness(sets, current_state), counter
            




In [53]:
num_sets

100

In [76]:
hill_climbing(1000, 1000, 0.6)

 The fitness of the initial state is :  (0, 0)


((1000, -480), 100000)